# This is a possible solution for the Week 1 labo made in pytorch

## The problem

Given is a dataset of Iris flowers. Each flower is described by 4 features: sepal length, sepal width, petal length, and petal width. The goal is to classify the flowers into one of three species: Iris setosa, Iris versicolor, or Iris virginica.

### Importing the dataset

In [1]:
# Import scikit-learn dataset library
from sklearn import datasets

# Load dataset
iris = datasets.load_iris()

### Exploring the dataset

In [2]:
# print the names of the 4 features
print("Features: ", iris.feature_names)

# print the label type of iris
print("Labels: ", iris.target_names)

# print data(feature)shape
print("Data shape: ", iris.data.shape)

# print the iris data features (top 5 records)
print(iris.data[0:5])

# print the iris labels (0:setosa, 1:versicolor, 2:virginica)
print(iris.target)

# print the length of the iris dataset
print(len(iris.data))

Features:  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Labels:  ['setosa' 'versicolor' 'virginica']
Data shape:  (150, 4)
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
150


### Preprocessing the dataset

In [21]:
# split the dataset into training and test sets
from sklearn.model_selection import train_test_split

seed = 42

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=seed)  # 70% training

print(X_train.shape)

# Normalize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

(105, 4)


### Building the model

In [4]:
import torch as t
from torch import nn

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = t.relu(self.fc1(x))
        x = t.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Choising the loss function and optimizer

In [6]:
loss_function = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(NeuralNetwork().parameters(), lr=0.01)

### Training the model

In [37]:
# Train the model

amount_of_epochs = 1000

for epoch in range(amount_of_epochs):
    optimizer.zero_grad() # clear the gradients
    outputs = NeuralNetwork()(t.tensor(X_train).float()) # forward pass, makes sure the input is a float tensor
    loss = loss_function(outputs, t.tensor(y_train)) # calculate the loss
    loss.backward() # backward pass
    optimizer.step() # update weights

    if epoch % 100 == 0:
        print(f'Epoch {epoch} - Loss: {loss.item()}')

Epoch 0 - Loss: 1.0817012786865234
Epoch 100 - Loss: 1.1121052503585815
Epoch 200 - Loss: 1.1202964782714844
Epoch 300 - Loss: 1.0727903842926025
Epoch 400 - Loss: 1.2573022842407227
Epoch 500 - Loss: 1.2051637172698975
Epoch 600 - Loss: 1.137162685394287
Epoch 700 - Loss: 1.0995595455169678
Epoch 800 - Loss: 1.5541194677352905
Epoch 900 - Loss: 1.098450779914856


### Evaluating the model

In [ ]:
# Test the model

with t.no_grad(): #
    correct = 0
    total = 0
    outputs = NeuralNetwork()(t.tensor(X_test).float())
    for idx, i in enumerate(outputs):
        if t.argmax(i) == t.tensor(y_test[idx]):
            correct += 1
        total += 1

    print('Accuracy: ', round(correct / total, 3))

Accuracy:  0.422


As you can see the accuracy of our modl is only 0.311. This is because we have a very small dataset and we are not using any advanced techniques to improve the model.

### Saving the model

While working on a project, you might want to save your model so that you can use it later. You can save the model using the `torch.save` function.

In [28]:
# Save the model

t.save(NeuralNetwork().state_dict(), 'iris_model.pth')

### Loading the model

Loading the model and making predictions is very easy. You can load the model using the `torch.load` function and then use the model to make predictions.

In [32]:
# Load the model

model = NeuralNetwork()
model.load_state_dict(t.load('iris_model.pth'))


/tmp/ipykernel_6598/3603961524.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load('iris_model.pth'))


<All keys matched successfully>

In [34]:
random_iris = t.tensor([[5.1, 3.5, 1.4, 0.2]]) # setosa

prediction = model(random_iris.float())
print(prediction)
print(t.argmax(prediction))


tensor([[-0.3971, -0.5863,  0.5146]], grad_fn=<AddmmBackward0>)
tensor(2)
